In [36]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cftime
import os

# So this code is so ghetto. Anna wrote some of it, Sarah wrote some of it, and I tried to make sense of it all. 
# The sea ice data is in different years so I hard coded the files and ran them individually for VRS and NYA. 
# I know there is definitely better way to do this but again I am not a python person. 

# path = 'siconc_SIday_NICAM16-7S_highresSST-present_r1i1p1f1_gr_20110101-20111231.nc'
# sea_ice11 = xr.open_dataset(path)
# sea_ice11['siconc'] = sea_ice11['siconca']
# sea_ice11.drop('siconca')
# sea_ice11=sea_ice11.drop('siconca')

# path1 = 'siconc_SIday_NICAM16-7S_highresSST-present_r1i1p1f1_gr_20120101-20121231.nc'
# sea_ice12 = xr.open_dataset(path1)

# path2 = 'siconc_SIday_NICAM16-7S_highresSST-present_r1i1p1f1_gr_20130101-20131231.nc'
# sea_ice13 = xr.open_dataset(path2)

path3 = 'siconc_SIday_NICAM16-7S_highresSST-present_r1i1p1f1_gr_20140101-20141231.nc'
sea_ice14 = xr.open_dataset(path3)

# sea_ice_all=xr.merge([sea_ice11,sea_ice12,sea_ice13,sea_ice14])



In [37]:
pathname='shared-ns1000k/dataporten-home/b8b982ef-2de983-2d4813-2dbbd2-2d5e4e04927306/'
match_date_file = './AMDE_NYA.txt'
date_data = pd.read_csv(match_date_file, header = None)
date_data.columns = ['Date/Time']
date_data['Date/Time'] = pd.to_datetime(date_data['Date/Time'])
date_data = date_data.set_index('Date/Time')

In [38]:
import sys 
sys.path.append('negi-stuff/')
from negi_stuff.modules import make_folders

In [39]:
def get_wghts(lat):
    """
    get latitude weights for gaussian grid.
    :param lat: latitudes
    :return: weights
    """
    latr = np.deg2rad(lat) # convert to radians
    weights = np.cos(latr) # calc weights
    return weights


In [40]:
def masked_average(xa:xr.DataArray,
                   dim=None,
                   weights:xr.DataArray=None,
                   mask:xr.DataArray=None):
    """
    This function will average
    :param xa: dataArray
    :param dim: dimension or list of dimensions. e.g. 'lat' or ['lat','lon','time']
    :param weights: weights (as xarray)
    :param mask: mask (as xarray), True where values to be masked.
    :return: masked average xarray
    """
    #lest make a copy of the xa
    xa_copy:xr.DataArray = xa.copy()

    if mask is not None:
        xa_weighted_average = __weighted_average_with_mask(
            dim, mask, weights, xa, xa_copy
        )
    elif weights is not None:
        xa_weighted_average = __weighted_average(
            dim, weights, xa, xa_copy
        )
    else:
        xa_weighted_average =  xa.mean(dim)

    return xa_weighted_average



    # %% [markdown]


def __weighted_average(dim, weights, xa, xa_copy):
    '''helper function for masked_average'''
    _, weights_all_dims = xr.broadcast(xa, weights)  # broadcast to all dims
    x_times_w = xa_copy * weights_all_dims
    xw_sum = x_times_w.sum(dim)
    x_tot = weights_all_dims.where(xa_copy.notnull()).sum(dim=dim)
    xa_weighted_average = xw_sum / x_tot
    return xa_weighted_average


def __weighted_average_with_mask(dim, mask, weights, xa, xa_copy):
    '''helper function for masked_average'''
    _, mask_all_dims = xr.broadcast(xa, mask)  # broadcast to all dims
    xa_copy = xa_copy.where(np.logical_not(mask))
    if weights is not None:
        _, weights_all_dims = xr.broadcast(xa, weights)  # broadcast to all dims
        weights_all_dims = weights_all_dims.where(~mask_all_dims)
        x_times_w = xa_copy * weights_all_dims
        xw_sum = x_times_w.sum(dim=dim)
        x_tot = weights_all_dims.where(xa_copy.notnull()).sum(dim=dim)
        xa_weighted_average = xw_sum / x_tot
    else:
        xa_weighted_average = xa_copy.mean(dim)
    return xa_weighted_average

def get_wghts(lat):
    """
    get latitude weights for gaussian grid.
    :param lat: latitudes
    :return: weights
    """
    latr = np.deg2rad(lat) # convert to radians
    weights = np.cos(latr) # calc weights
    return weights


In [41]:
path='temp/sea_ice_slice_vrs_14.nc'
make_folders.make_folders(path)
# Lat for NYA 79 Lon for NYA 12
# Lat for VRS 81 Lon for VRS -16
# We say plus minus 4 deg
sea_ice_slice=sea_ice14.sel(lat=slice(77, 85), lon=slice(340,348)).to_netcdf(path)

In [42]:
path='temp/sea_ice_slice_vrs_14.nc'

sea_ice_slice_wrking = xr.open_dataset(path)
#sea_ice_slice_wrking['siconc'] = sea_ice_slice_wrking['siconca']
# sea_ice_slice_wrking.drop('siconca')
#sea_ice_slice_wrking.siconc

In [46]:
# #aw_xr = areacella['areacella']
glob_mean_si = masked_average(sea_ice_slice_wrking['siconc'], dim=['lat','lon'], weights=get_wghts(sea_ice_slice_wrking['lat']))

#glob_mean_si
glob_mean_si.to_netcdf('temp/glob_mean_si14_vrs.nc')

#glob_mean_si

In [47]:
# # glob_mean_si_11
a=xr.open_dataset('temp/glob_mean_si11_vrs.nc')
b=xr.open_dataset('temp/glob_mean_si12_vrs.nc')
c=xr.open_dataset('temp/glob_mean_si13_vrs.nc')
d=xr.open_dataset('temp/glob_mean_si14_vrs.nc')
e=xr.merge((a,b))
f=xr.merge((e,c))
glob_mean_si_vrs_11_14=xr.merge((f,d))
glob_mean_si_vrs_11_14
# .to_netcdf('NYA_SI_11_14.nc')
# mean_si_11_14_nya=xr.open_dataset('NYA_SI_11_14.nc')

<xarray.Dataset>
Dimensions:                        (time: 1461)
Coordinates:
  * time                           (time) datetime64[ns] 2011-01-01T12:00:00 ... 2014-12-31T12:00:00
    type                           |S7 b'sea_ice'
Data variables:
    __xarray_dataarray_variable__  (time) float64 99.24 99.25 ... 99.5 99.49

In [49]:
glob_mean_si_vrs_11_14['SI']=glob_mean_si_vrs_11_14['__xarray_dataarray_variable__']
glob_mean_si_vrs_11_14.drop('__xarray_dataarray_variable__')

<xarray.Dataset>
Dimensions:  (time: 1461)
Coordinates:
  * time     (time) datetime64[ns] 2011-01-01T12:00:00 ... 2014-12-31T12:00:00
    type     |S7 b'sea_ice'
Data variables:
    SI       (time) float64 99.24 99.25 99.25 99.26 ... 99.5 99.5 99.5 99.49

In [90]:
si=xr.Dataset({'SI': (('time'),mean_si_11_14_nya)},
                {'time': mean_si_11_14_nya.time}
               )

# si.to_netcdf('SI_11_14_NYA.nc')

ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): ('time', <xarray.Dataset>
Dimensions:                        (time: 1461)
Coordinates:
  * time                           (time) datetime64[ns] 2011-01-01T12:00:00 ... 2014-12-31T12:00:00
    type                           |S7 ...
Data variables:
    __xarray_dataarray_variable__  (time) float64 ...) to Variable.

In [70]:
dset=xr.open_dataset('SI_12_NYA.nc')

list_times = []
for t in date_data.index:
    list_times.append(cftime.DatetimeNoLeap(t.year, t.month, t.day, 0, 0, 0, 0, 0, 0))

In [75]:
AMDE_SI_NYA=dset.sel(time=date_data.index.tolist())

KeyError: "not all values found in index 'time'"

In [73]:
AMDE_SI_NYA

<xarray.Dataset>
Dimensions:  (time: 45)
Coordinates:
    type     |S7 ...
  * time     (time) datetime64[ns] 2012-01-01T12:00:00 ... 2012-12-31T12:00:00
Data variables:
    SI       (time) float64 ...

In [ ]:
# a=xr.open_dataset('temp/glob_mean_si12_NYA.nc')
# b=xr.open_dataset('temp/glob_mean_si13_NYA.nc')
# c=xr.open_dataset('temp/glob_mean_si14_NYA.nc')
# d=xr.open_dataset('temp/glob_mean_si11_NYA.nc')
# e=xr.merge((a,b))
# f=xr.merge((e,c))
# glob_mean_si_11_14=xr.merge((f,d))
# glob_mean_si_11_14

In [50]:
#AMDE_RAD_VRS.to_netcdf("AMDE_RAD_VRS.csv")
aaa = pd.DataFrame({'time': glob_mean_si_vrs_11_14.time, 'SI': glob_mean_si_vrs_11_14.SI})

aaa.to_csv('AMDE_SI_vrs_11_14.csv')